In [1]:
# Import Libraries
import pandas as pd
import georasters as gr
import pyproj as ppj
import math
import os
import sys

C:\Users\Danie\Anaconda3\lib\site-packages\pysal\explore\segregation\network\network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"
C:\Users\Danie\Anaconda3\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
import pyproj
pyproj.Proj("+init=epsg:4326")

pyproj.Proj('+units=m +init=epsg:4326', preserve_units=True)

In [2]:
# Declare folder
data_folder = os.path.abspath("F:\\FellowshipWork-Daniel.h\\NDVI Data\\CA\\2000\\2000 renamed")
output_folder = os.path.abspath("output")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:
#Read CSV file
csv_file = "CA 2000.csv"
csv_file = os.path.join(data_folder, csv_file)
#df = pd.read_csv(csv_file)
#tif name format: [year][month][week of year]
dtype_dic = {'DAY': str,
            'WEEK': str,
             'MONTH': str,
            'YEAR': str,
            'LATITUDE': 'float',
            'LONGITUDE': 'float'}
df = pd.read_csv(csv_file, dtype = dtype_dic)
df['Filename'] = (df['YEAR']).astype(str).str.slice(start=0, stop=None, step=None) + "_" + df['WEEK'].astype(str) + ".tif"#FILENAME POSSIBILTY WRONG?
#df['filename'] = (df['YEAR']).astype(str).str.slice(start=0, stop=None, step=None) + formattedMonth(df['MONTH']).astype(str) + df['DAY'].astype(str) + ".tif"
df

,DAY,WEEK,MONTH,YEAR,LATITUDE,LONGITUDE,DEM_Value,Aspect_Value,Slope_Value,Filename
0,06,02,01,2000,0.00000,0.000,<Null>,<Null>,<Null>,2000_02.tif
1,27,05,01,2000,33.51799,-116.173,50.64861,100.498,1.642063,2000_05.tif
2,31,06,01,2000,39.64752,-121.496,281.4343,328.0066,21.14061,2000_06.tif
3,02,06,02,2000,32.64200,-116.401,891.7449,215.4753,5.484125,2000_06.tif
4,04,06,02,2000,37.81800,-120.252,925.712,46.25621,14.26075,2000_06.tif
...,...,...,...,...,...,...,...,...,...,...
263,15,47,11,2000,34.52610,-115.589,202.3059,179.3844,0.498611,2000_47.tif
264,17,47,11,2000,32.61640,-116.823,536.8585,114.8332,17.98893,2000_47.tif
265,29,49,11,2000,37.45600,-118.335,1285.189,207.9969,4.260321,2000_49.tif
266,14,51,12,2000,35.19935,-119.506,334.0211,72.63743,6.059639,2000_51.tif


In [ ]:
#loop over filename column values to read files and extract value by latlong
def get_raster_value_at_point(row):#the hell does row['filename'] mean??
    filepath = os.path.join(data_folder, row['filename'])#works??
    #print(filepath)
    raster_data = gr.from_file(filepath)#create georaster object from .tif file, AttributeError when tif file doesn't exist
    #print(raster_data)
    #raster_projection = ppj.Proj(raster_data.projection.ExportToProj4())
    #csv_projection = ppj.Proj(init='epsg:4326')
    #csv_point_on_raster = ppj.transform(csv_projection, raster_projection, row['Longitude'], row['Latitude'], z=None, radians=False)
    #value = raster_data.map_pixel(csv_point_on_raster[0], csv_point_on_raster[1])
    #value = float(value)
    #if math.isnan(value):
    #    value = 0
    #print(value)
    #return value
df['value'] = df.apply(get_raster_value_at_point, axis=1)

In [4]:
#loop over filename column values to read files and extract value by latlong
def get_raster_value_at_point(row):
    filepath = os.path.join(data_folder, row['Filename'])#works??
    #print(filepath)
    try:
        raster_data = gr.from_file(filepath)#create georaster object from .tif file, AttributeError when tif file doesn't exist
        #print(raster_data)
        raster_projection = ppj.Proj(raster_data.projection.ExportToProj4())
        csv_projection = ppj.Proj(init='epsg:4326')
        csv_point_on_raster = ppj.transform(csv_projection, raster_projection, row['LONGITUDE'], row['LATITUDE'], z=None, radians=False)
        value = raster_data.map_pixel(csv_point_on_raster[0], csv_point_on_raster[1])
    except AttributeError:
        print ("Error: .tif file for week " + row['WEEK'] + " month " + row['MONTH'] + " not found. Skipping")
        value = 0
    value = float(value)
    if math.isnan(value):
        value = 0
    #print(value)
    return value
df['NDVI_Value'] = df.apply(get_raster_value_at_point, axis=1)

Error: .tif file for week 02 month 01 not found. Skipping
Error: .tif file for week 05 month 01 not found. Skipping
Error: .tif file for week 06 month 01 not found. Skipping
Error: .tif file for week 06 month 02 not found. Skipping
Error: .tif file for week 06 month 02 not found. Skipping
Error: .tif file for week 08 month 02 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif file for week 33 month 08 not found. Skipping
Error: .tif fi

In [5]:
df

,DAY,WEEK,MONTH,YEAR,LATITUDE,LONGITUDE,DEM_Value,Aspect_Value,Slope_Value,Filename,NDVI_Value
0,06,02,01,2000,0.00000,0.000,<Null>,<Null>,<Null>,2000_02.tif,0.0
1,27,05,01,2000,33.51799,-116.173,50.64861,100.498,1.642063,2000_05.tif,0.0
2,31,06,01,2000,39.64752,-121.496,281.4343,328.0066,21.14061,2000_06.tif,0.0
3,02,06,02,2000,32.64200,-116.401,891.7449,215.4753,5.484125,2000_06.tif,0.0
4,04,06,02,2000,37.81800,-120.252,925.712,46.25621,14.26075,2000_06.tif,0.0
...,...,...,...,...,...,...,...,...,...,...,...
263,15,47,11,2000,34.52610,-115.589,202.3059,179.3844,0.498611,2000_47.tif,801.0
264,17,47,11,2000,32.61640,-116.823,536.8585,114.8332,17.98893,2000_47.tif,6215.0
265,29,49,11,2000,37.45600,-118.335,1285.189,207.9969,4.260321,2000_49.tif,868.0
266,14,51,12,2000,35.19935,-119.506,334.0211,72.63743,6.059639,2000_51.tif,1900.0


In [6]:
df.to_csv(os.path.join(output_folder, "CA_NDVI_2000_V2.csv"), encoding='utf-8', index=False)